In [24]:
from incubator26.yolo import loss, model
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as layers
import keras.backend as K

In [20]:
y_pred = tf.convert_to_tensor(np.random.rand(16, 16, 16, 5))
y_true = tf.convert_to_tensor(np.random.rand(16, 16, 16, 5))

In [22]:
def yolo_loss(y_true, y_pred):
    # define variables from input tensors y_true and y_pred with shape (None, 32, 32, 5) respectively

    y_pred_box_prob = y_pred[:, :, :, 0]  # shape = (None, 32, 32, 1)
    y_pred_x = y_pred[:, :, :, 1]  # shape = (None, 32, 32, 1)
    y_pred_y = y_pred[:, :, :, 2]  # shape = (None, 32, 32, 1)
    y_pred_w = y_pred[:, :, :, 3]  # shape = (None, 32, 32, 1)
    y_pred_h = y_pred[:, :, :, 4]  # shape = (None, 32, 32, 1)

    y_true_box_prob = y_true[:, :, :, 0]  # shape = (None, 32, 32, 1)
    y_true_x = y_true[:, :, :, 1]  # shape = (None, 32, 32, 1)
    y_true_y = y_true[:, :, :, 2]  # shape = (None, 32, 32, 1)
    y_true_w = y_true[:, :, :, 3]  # shape = (None, 32, 32, 1)
    y_true_h = y_true[:, :, :, 4]  # shape = (None, 32, 32, 1)

    # define response mask: object in grid, yes or no?

    response_mask = y_true[:, :, :, 0]  # shape = (None, 32, 32, 1)

    # set value for lamba_coord (default from paper = 5) and lambda_noobj (default from paper = 0.5)
    lambda_coord = 5
    lambda_noobj = 0.5

    # calculate sum of squared x and y differences
    x_diff = y_true_x - y_pred_x
    y_diff = y_true_y - y_pred_y

    x_and_y_loss = lambda_coord * response_mask * (K.square(x_diff) + K.square(y_diff))

    # calculate sum of squared w and h differences
    w_diff = K.sqrt(y_true_w) - K.sqrt(y_pred_w)
    h_diff = K.sqrt(y_true_h) - K.sqrt(y_pred_h)

    w_and_h_loss = lambda_coord * response_mask * (K.square(w_diff) + K.square(h_diff))

    # calculate sum of squared box prob differences
    box_prob_diff = y_true_box_prob - y_pred_box_prob
    no_object_loss = lambda_noobj * (1 - response_mask) * K.square(0 - y_pred_box_prob)
    object_loss = response_mask * K.square(1 - y_pred_box_prob)

    box_prob_loss = no_object_loss + object_loss

    # calculate yolo loss
    yolo_loss = K.sum(x_and_y_loss + w_and_h_loss + box_prob_loss)

    return yolo_loss

In [23]:
yolo_loss(y_true, y_pred)

<tf.Tensor: shape=(), dtype=float64, numpy=6800.033794272225>